In [1]:
!pip install -q google-generativeai==0.3.1

In [2]:
import google.generativeai as genai
from pathlib import Path
import sqlite3

In [5]:
genai.__version__

'0.3.1'

In [7]:
import json


with open('config.json') as f:
    userdata = json.load(f)

genai.configure(api_key=userdata.get('Generative_Language_Client'))

In [9]:
generation_config = {
  "temperature": 0.4,
  "top_p": 1,
  "top_k": 32,
  "max_output_tokens": 4096,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  }
]

In [11]:
model = genai.GenerativeModel(model_name = "gemini-pro",
                              generation_config = generation_config,
                              safety_settings = safety_settings)

In [13]:

def read_sql_query(sql, db):
    conn = sqlite3.connect(db)
    cur = conn.cursor()
    cur.execute(sql)
    rows = cur.fetchall()
    for row in rows:
        print(row)
    conn.close()


In [15]:
read_sql_query("SELECT department, COUNT(*) AS issue_count\nFROM it_tickets\nGROUP BY department\nORDER BY issue_count DESC\nLIMIT 1;",
               "it_tickets_db.sqlite")

('IT Support', 33)


In [17]:
# prompt_parts_1 = [
#   "You are an expert in converting English questions to SQL code! The SQL database has the name fashion_products and has the following columns - user_id, product_id, product_name, brand, category, price, color, and size.\n\nFor example,\nExample 1 - How many entries of Adidas are present?, the SQL command will be something like this\n``` SELECT COUNT(*) FROM fashion_products WHERE brand = 'Adidas';\n```\n\nExample 2 - How many XL products of Nike are there that have a rating of more than 4?\n```\nSELECT COUNT(*) FROM fashion_products WHERE brand = 'Nike' AND size = 'XL' AND \"Rating\" > 4;\n```\n\nExample 3 - \n```\nSELECT product_name FROM fashion_products WHERE price = (SELECT MAX(price) FROM fashion_products);\n```\n\nDont include ``` and \n in the output",
# ]

prompt_parts_1 = [
    "You are an expert in converting English questions to SQL code! The SQL database has the name it_tickets and has the following columns - ticket_id, date_opened, issue_type, priority, assigned_to, status, date_closed, department, and description.\n\nFor example,\nExample 1 - How many open tickets are there?, the SQL command will be something like this\nSELECT COUNT(*) FROM it_tickets WHERE status = 'Open';\n\nExample 2 - How many high-priority tickets have been assigned to John Doe?\nSELECT COUNT(*) FROM it_tickets WHERE priority = 'High' AND assigned_to = 'John Doe';\n\nExample 3 - Which ticket has been open for the longest time?\nSELECT ticket_id FROM it_tickets WHERE date_opened = (SELECT MIN(date_opened) FROM it_tickets) AND status = 'Open';\n\nExample 4 - How many tickets were closed in the Finance department?\nSELECT COUNT(*) FROM it_tickets WHERE department = 'Finance' AND status = 'Closed';\n\nExample 5 - List the description of tickets assigned to Sarah Chen that are still in progress?\nSELECT description FROM it_tickets WHERE assigned_to = 'Sarah Chen' AND status = 'In Progress';\n\nExample 6 - How many tickets have been closed with a priority of 'Low'?\nSELECT COUNT(*) FROM it_tickets WHERE status = 'Closed' AND priority = 'Low';\n\nExample 7 - What are the departments with the most number of high-priority tickets?\nSELECT department, COUNT(*) FROM it_tickets WHERE priority = 'High' GROUP BY department ORDER BY COUNT(*) DESC;\n\nExample 8 - Which tickets have been assigned to both John Doe and Michael Lee at some point?\nSELECT ticket_id FROM it_tickets WHERE assigned_to IN ('John Doe', 'Michael Lee') GROUP BY ticket_id HAVING COUNT(DISTINCT assigned_to) = 2;\n\nExample 9 - What is the average time taken to close high-priority tickets?\nSELECT AVG(JULIANDAY(date_closed) - JULIANDAY(date_opened)) AS avg_time FROM it_tickets WHERE priority = 'High' AND status = 'Closed';\n\nExample 10 - List all tickets that were opened before March 2024 but are still not closed?\nSELECT * FROM it_tickets WHERE date_opened < '2024-03-01' AND status != 'Closed';"
]


In [19]:
question = "Tell me the most issue facing department ?"

In [21]:
prompt_parts = [prompt_parts_1[0], question]
response = model.generate_content(prompt_parts)
response.text

'```sql\nSELECT department, COUNT(*) AS issue_count\nFROM it_tickets\nGROUP BY department\nORDER BY issue_count DESC\nLIMIT 1;\n```'

In [23]:
read_sql_query("SELECT department, COUNT(*) AS issue_count\nFROM it_tickets\nGROUP BY department\nORDER BY issue_count DESC\nLIMIT 1;",
               "it_tickets_db.sqlite")

('IT Support', 33)


In [39]:
def generate_gemini_response(question, input_prompt):
    prompt_parts = [input_prompt, question]

    response = model.generate_content(prompt_parts)

    query = response.text.strip().replace('```sql', '').replace('```', '')

    print(f"Interpreted Query: {question}")
    
    print(f"Technical Filter (SQL Query):\n{query}")

    refine = input("Would you like to refine the SQL query? (yes/no): ").lower()
    
    if refine == 'yes':
        query = input("Please enter the adjusted SQL query: ")

    output = read_sql_query(query, "it_tickets_db.sqlite")

    return output

In [37]:
generate_gemini_response("List the descriptions of tickets assigned to Sarah Chen that are still in progress ?",
                         prompt_parts_1[0])

Interpreted Query: List the descriptions of tickets assigned to Sarah Chen that are still in progress ?
Technical Filter (SQL Query):

SELECT description
FROM it_tickets
WHERE assigned_to = 'Sarah Chen'
AND status = 'In Progress';



Would you like to refine the SQL query? (yes/no):  no


('Printer not working',)
('Unable to access database',)
('Need new software installed',)
('Wi-Fi connection unstable',)
('Wi-Fi connection unstable',)
('Wi-Fi connection unstable',)
('Unable to log into system',)
<class 'NoneType'>
